In [ ]:
import numpy as np
import re, string, unicodedata, nltk
from nltk import word_tokenize, sent_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from gensim.models.coherencemodel import CoherenceModel
import re, string, unicodedata
import matplotlib.pyplot as plt
#import dask.dataframe as dd

In [1]:
import pandas as pd
#import dask.dataframe as ddf
confessions = pd.read_csv("one-million-reddit-confessions.csv",usecols=["selftext", "title"])
#confessions = dd.read_csv("one-million-reddit-confessions.csv")
#confessions =confessions[["selftext", "title"]]

In [2]:
confessions

,selftext,title
0,\n\nI've always been great with children. I u...,I was horrible to children at my first prescho...
1,[removed],I asked my crush for sending me her picture an...
2,[removed],I'm a fucking dunce and want to cringe myself ...
3,"Hello everyone, I'm probably going to be made ...",I have 0 friends and now I fantasies about hav...
4,[removed],I hired a escort and I can’t live this with re...
...,...,...
999995,I always wonder what side of their pants that ...,When i see men...
999996,[deleted],I hate being a 5’9 guy
999997,[deleted],The only social life I really have is reddit.
999998,BLUF: I use to avoid doing/using things that r...,Does anyone do things/use items that remind th...


## Preprocessing

In [3]:
# remove empty values
confessions = confessions.dropna()
# remove repeated rows
confessions = confessions.drop_duplicates()

In [4]:
# remove unrelated information
confessions = confessions[confessions['selftext'] != '[removed]']
confessions = confessions[confessions['selftext'] != '[deleted]']
confessions = confessions.reset_index(drop=True)

In [5]:
# put title and main text together to give more information
confessions['Content'] = confessions['title'] + " " + confessions['selftext']

In [ ]:
#confessions_dask = ddf.from_pandas(confessions[['Content']], npartitions=10)

In [ ]:
#confessions_dask

In [6]:
# remove emojis, greek letters, accents...
confessions['Content'] = confessions['Content'] .str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
#confessions['Content'] =confessions['Content'] .astype(str)

In [ ]:
"""def to_lowercase(df):
    #df["Content"] = df.Content.map(lambda text:text.lower())#.map(remove_emails).map(remove_newlinechars).map(remove_stopwords).map(lemmatize)
    df["Content"] = df['Content'].str.lower()
    return df

def remove_shorter_words(df):
    df["Content"] = df.Content.map(lambda Content: ' '.join([t for t in Content.split() if len(t)>3]))#.map(remove_emails).map(remove_newlinechars).map(remove_stopwords).map(lemmatize)
    return df

stop_words = stopwords.words('english')
stop_words.extend(['hello','everyone','couldv','couldnt','doesnt','havent','year','wasnt','isnt','also','done','although','alot','kinda','edit','dont','didnt','would','really','shit','cant','fucking'])

def remove stop_words(df):
    tokenized_doc = confessions['Content'].map(lambda x: x.split())
    filtered = filter(lambda word: word not in stop_words, words)
    return list(filtered)"""

In [ ]:
"""class cleantext():
    
    def __init__(self, text = "test"):
        self.text = text
    
    def get_words(self):
        self.words = nltk.word_tokenize(self.text)
        return self

    def remove_non_ascii(self):
        #Remove non-ASCII characters from list of tokenized words
        new_words = []
        for word in self.words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        self.words = new_words
        return self

    def to_lowercase(self):
        # Convert all characters to lowercase from list of tokenized words
        new_words = []
        for word in self.words:
            new_word = word.lower()
            new_words.append(new_word)
        self.words = new_words
        return self

    def remove_punctuation(self):
        #Remove punctuation from list of tokenized words
        new_words = []
        punctuations = string.punctuation
        for word in self.words:
            if word not in punctuations:
                new_words.append(word)
        self.words = new_words
        return self


    def remove_stopwords(self):
        #Remove stop words from list of tokenized words
        stop_words = stopwords.words('english')
        stop_words.extend(['hello','everyone','couldv','couldnt','doesnt','havent','year','wasnt','isnt','also','done','although','alot','kinda','edit','dont','didnt','would','really','shit','cant','fucking'])
        new_words = []
        for word in self.words:
            if word not in stop_words:
                new_words.append(word)
        self.words = new_words
        return self


    def lemmatize_verbs(self):
        #Lemmatize verbs in list of tokenized words
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for word in self.words:
            lemma = lemmatizer.lemmatize(word, pos='v')
            lemmas.append(lemma)
        self.words = lemmas
        return self
    
    def join_words(self):
        self.words = ' '.join(self.words)
        return self
    
    def do_all(self, text):
        
        self.text = text
        self = self.get_words()
        self = self.remove_punctuation()
        self = self.remove_non_ascii()
        self = self.remove_stopwords()
        self = self.lemmatize_verbs()
        self = self.join_words()
        
        return self.words"""

In [7]:
# remove_punctuation
import string
punctuations = string.punctuation
confessions['Content'] = confessions['Content'].apply(lambda x: "".join([i for i in x if i not in punctuations]))

In [8]:
# remove short words
confessions['Content'] = confessions['Content'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
#confessions_dask = confessions_dask.map_partitions(to_lowercase)

In [ ]:
#ct = cleantext(confessions_dask['Content'])

In [ ]:
#def dask_this(df):
#    res = df['Content'].apply(ct.do_all)
#    return res  
#res = confessions_dask.map_partitions(dask_this, meta=confessions)

In [ ]:
#confessions_df = res.compute()

In [ ]:
#confessions_dask = confessions_dask.map_partitions(remove_shorter_words)

In [ ]:
#confessions_dask = confessions_dask.map_partitions(stop_words, meta=confessions)

In [9]:
# make all text lowercase
confessions['Content']= confessions['Content'].str.lower()

In [20]:
confessions

,selftext,title,Content
0,\n\nI've always been great with children. I u...,I was horrible to children at my first prescho...,horrible children first preschool always been ...
1,"Hello everyone, I'm probably going to be made ...",I have 0 friends and now I fantasies about hav...,have friends fantasies about having friends he...
2,My son is 20 and he does have a sex life. I as...,I set up cameras in my son's apartment and wat...,cameras sons apartment watched have does have ...
3,I did one of the most immoral things that I co...,I drove while a little high and it's something...,drove while little high something always promi...
4,So for a long time I had a problem with steali...,I stole $200 from my little sister to go shopp...,stole from little sister shopping lied about l...
...,...,...,...
427462,"One of my closest friends, who I will call Eri...",I'm the only person my best friend trusts to s...,only person best friend trusts share mental he...
427463,# Diaraeah maNNN\n\n*sHItttttTing my pantZ*\n\...,dfiareahh MannN💩 💩💩😎😎 ({[ teh sexcond verse)))...,dfiareahh mannn sexcond verse diaraeah mannn s...
427464,I always wonder what side of their pants that ...,When i see men...,when always wonder what side their pants that ...
427465,BLUF: I use to avoid doing/using things that r...,Does anyone do things/use items that remind th...,does anyone thingsuse items that remind them t...


In [18]:
#confessions["Content"] = confessions["Content"].astype(str)

In [11]:
from nltk.corpus import stopwords

In [12]:
# remove stop words
stop_words = stopwords.words('english')
stop_words.extend(["could","edit2","thats","wouldnt","shouldnt","shant","neednt","mustnt","hadnt","hadnt","couldve","arent","thatll","she's","youd","youll","youve","youre",'hello',"mightnt",'ive','everyone','couldv','couldnt','doesnt','havent',"wont",'wasnt','isnt',"werent",'also','done','although','alot','kinda','edit','dont','didnt','would','really','shit','cant','fucking'])
tokenized_doc = confessions['Content'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [15]:
tokenized_doc

0         [horrible, children, first, preschool, always,...
1         [friends, fantasy, friends, probably, go, make...
2         [cameras, sons, apartment, watch, life, detail...
3         [drive, little, high, something, always, promi...
4         [steal, little, sister, shop, lie, long, time,...
                                ...                        
427462    [person, best, friend, trust, share, mental, h...
427463    [dfiareahh, mannn, sexcond, verse, diaraeah, m...
427464    [always, wonder, side, pant, dicks, rest, natu...
427465    [anyone, thingsuse, items, remind, bluf, avoid...
427466                                 [wait, inlaws, feel]
Name: Content, Length: 427467, dtype: object

In [14]:
# Lemmatization words
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
tokenized_doc = tokenized_doc.apply(lambda x: [wordnet_lemmatizer.lemmatize(i,pos="v") for i in x])

In [21]:
confessions.to_csv("confessions_cleaned.csv",index=False)

In [ ]:
tokenized_doc.to_csv("tokenized.csv", index=False)

In [17]:
from gensim import corpora
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(tokenized_doc)

In [20]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_doc]

In [25]:
#corpora.MmCorpus.serialize('doc_term_matrix.mm',doc_term_matrix)

In [26]:
#corpus_load = corpora.MmCorpus('doc_term_matrix.mm')

In [29]:
#for line in corpus_load:
#    print(line)

In [22]:
print(len(dictionary))
print(len(doc_term_matrix))

291315
427467


In [30]:
#from gensim.models import TfidfModel

In [31]:
#model = TfidfModel(doc_term_matrix)  # fit model

In [36]:
#tfidf_model = model[doc_term_matrix]  # apply model to the corpus document

In [55]:
num_topics = 20
chunksize = 5000
#passes = 20
#iterations = 400
eval_every = None

In [51]:
temp = dictionary[0] 

In [53]:
id2word = dictionary.id2token

In [57]:
from gensim.models import LdaModel

In [58]:
model = LdaModel(
    corpus=doc_term_matrix,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    #iterations=iterations,
    num_topics=num_topics,
    #passes=passes,
    eval_every=eval_every)

In [2]:
#from gensim.models import LsiModel

In [59]:
# generate LSA model
#lsamodel = LsiModel(doc_term_matrix, num_topics=n, id2word = dictionary)  # train model
#print(lsamodel.print_topics(num_topics=n, num_words=words))

In [60]:
top_topics = model.top_topics(doc_term_matrix)

In [63]:
from gensim.test.utils import datapath
temp_file = datapath("ldamodel")
model.save(temp_file)

In [ ]:
#lda = LdaModel.load(temp_file)

In [61]:
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

Average topic coherence: -1.8379.


In [ ]:
#vectorizer = TfidfVectorizer(stop_words=none,max_features=1000, max_df = 0.99, use_idf = True, ngram_range=(1,3))

In [42]:
#import matplotlib.pyplot as plt

In [45]:
#X = vectorizer.fit_transform(test)

In [44]:
#plt.spy(tfidf_model,markersize=1)

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/scale.html
# https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html
# https://examples.dask.org/machine-learning/text-vectorization.html